## Goal
Predict how long it takes for colleges to implement vaccine requirement. This notebook also includes preprocessing to be used in all future analyses with these data.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Preprocessing and Feature Engineering
Some was done in previous notebook, but most will be done here. I'm using the cleaned vaccine data from my 'Vaccine Mandates' notebook.

In [2]:
vacc_data = pd.read_csv('vacc_mandates_cleaned_school.csv')
vacc_data.head()

,College,ranking,state,city,zip,announce_date,Type,State_x,all_employee_vacc,some_employee_vacc,...,Region,Division,zip_str,cleaned_name_list,2020.student.size,school.name,school.zip,id,cleaned_school.name_list,name_similarity
0,Adelphi University,171,NY,Garden City,11530,116.0,Private,NY,0,0,...,Northeast,Middle Atlantic,11530,['adelphi'],5076.0,Adelphi University,11530,188429.0,['adelphi'],1.000000
1,American University,78,DC,Washington,20016,13.0,Private,DC,1,0,...,South,South Atlantic,20016,['american'],7510.0,American University,20016,131159.0,['american'],1.000000
2,Arizona State University,116,AZ,Tempe,85287,197.0,Public,AZ,1,0,...,West,Mountain,85287,"['arizona', 'state']",62633.0,Arizona State University Campus Immersion,85287,104151.0,"['arizona', 'state', 'immersion']",0.816497
3,Aurora University,302,IL,Aurora,60506,139.0,Private,IL,1,0,...,Midwest,East North Central,60506,['aurora'],4123.0,Aurora University,60506,143118.0,['aurora'],1.000000
4,Bellarmine University,202,KY,Louisville,40205,NaN,Private,KY,1,0,...,South,East South Central,40205,['bellarmine'],2431.0,Bellarmine University,40205,156286.0,['bellarmine'],1.000000


First, drop unnecessary columns (i.e., ones that won't be used as features in my model). Drop names, locations (as I extracted data from them) and ```state_pol```, as that was from the Chronicle data but I found different political data myself. Also dropping records of employee vaccination and boosters, as I'm trying to predict original student vaccination. I will consider ```all_students_vacc``` instead of ```res_students_vacc``` as it'll be less dependent on each college's dorm situations. Also drop ```Division``` as I don't have that much data, so using the more broad category of ```Region``` will probably be better.

In [3]:
vacc_data.drop(columns=['College', 'state', 'city', 'zip', 'State_x', 'state_fips', 
                        'STCOUNTYFP', 'county_fips', 'county_fips_str', 'state_pol', 
                        'State_y', 'State Code', 'Division', 'zip_str', 'cleaned_name_list',
                        'school.name', 'school.zip', 'id', 'cleaned_school.name_list', 'name_similarity',
                        'all_employee_vacc', 'some_employee_vacc', 'res_students_vacc'], 
               inplace=True)

In [4]:
vacc_data.head()

,ranking,announce_date,Type,all_students_vacc,booster,median_income,total_population,avg_hhsize,avg_community_level,political_control_state,county_vote_diff,Region,2020.student.size
0,171,116.0,Private,1,0,47254.0,1355683.0,2.62,0.624352,Dem,0.059304,Northeast,5076.0
1,78,13.0,Private,1,1,52328.0,701974.0,2.19,0.726562,Dem,0.867763,South,7510.0
2,116,197.0,Public,0,0,34032.0,4412779.0,2.64,0.492925,Rep,-0.028354,West,62633.0
3,302,139.0,Private,1,0,35433.0,531756.0,2.81,0.568831,Dem,0.105015,Midwest,4123.0
4,202,NaN,Private,1,0,32123.0,768419.0,2.25,1.062827,Div,0.133300,South,2431.0


As rankings are tied from 299-391 (see "Vaccine Mandates.ipynb") and a change in ranking won't usually signal a large change in a college's operations unless they move to a new echelon, I will place the rankings into bins. The rankings are fairly arbitrary besides 299-391. I isolated the top 20 because they often are spoken of in a different way. Then, I separated the rest of the top 100 from above 100 (another arbitrary cut, but one that makes sense when talking about top colleges).

In [5]:
vacc_data['ranking'] = pd.cut(vacc_data['ranking'], bins=[0, 20, 100, 200, 298, 400], labels=['a', 'b', 'c', 'd', 'e'], right=False)

Now, I need to figure out my target variable. Explore the two possibilities--categorical variable for vaccination status or continuous variable for days after first announcement.

In [6]:
vacc_data['all_students_vacc'].value_counts()

1    141
0     12
Name: all_students_vacc, dtype: int64

In [7]:
vacc_data[['all_students_vacc', 'booster']].value_counts()

all_students_vacc  booster
1                  0          99
                   1          42
0                  0          12
dtype: int64

In [8]:
vacc_data['announce_date'].isna().sum()

14

In [9]:
vacc_data.dropna(subset=['announce_date'], inplace=True)

With only 14 colleges not requiring the vaccine, there's likely not enough data to train a good model. So, I can drop these and do a regression estimating how long it takes for a college to institute a vaccination requirement, under the assumption that they will make one. This won't be as useful, but can help gauge each college's decision times and urgency regarding their requirements.

Note that in the future, I can also try to predict if a college that already had a vaccine requirement will require a booster. Svae this for later.

In [10]:
target_date = vacc_data['announce_date'] # Note: is continuous
features_date = vacc_data.drop(columns=['announce_date', 'all_students_vacc', 'booster'])
target_booster = vacc_data['booster'] # Note: is binary
features_booster = vacc_data.drop(columns=['all_students_vacc', 'booster'])

In [198]:
target_booster.to_pickle('target_booster.pkl')
features_booster.to_pickle('features_booster.pkl')

Now make sure all the vars are the correct type.

In [11]:
vacc_data.dtypes

ranking                    category
announce_date               float64
Type                         object
all_students_vacc             int64
booster                       int64
median_income               float64
total_population            float64
avg_hhsize                  float64
avg_community_level         float64
political_control_state      object
county_vote_diff            float64
Region                       object
2020.student.size           float64
dtype: object

In [12]:
vacc_data.columns

Index(['ranking', 'announce_date', 'Type', 'all_students_vacc', 'booster',
       'median_income', 'total_population', 'avg_hhsize',
       'avg_community_level', 'political_control_state', 'county_vote_diff',
       'Region', '2020.student.size'],
      dtype='object')

## Predicting Dates

Next, use one-hot encoding for the categorical variables. Note that my preprocessing is copied/based on [this sklearn course](https://inria.github.io/scikit-learn-mooc/python_scripts/03_categorical_pipeline_column_transformer.html).

In [16]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
categorical_preprocessor = OneHotEncoder(drop='first') # drop to avoid multicollinearity
numerical_preprocessor = StandardScaler() # normalize data to make it easier for sklearn models to handle

Testing on one column.

In [17]:
type_encoded = categorical_preprocessor.fit_transform(vacc_data[['Type']])
print(type_encoded.toarray()[:5])

[[0.]
 [0.]
 [1.]
 [0.]
 [0.]]


With a pipeline, apply to all categorical features. Also, standardize numerical features.

In [18]:
categorical_columns = ['ranking', 'Type', 'political_control_state', 'Region']
numerical_columns = list(set(features_date.columns).difference(categorical_columns))

In [19]:
from sklearn.compose import ColumnTransformer # splits the column, transforms each subset differently, then concatenates
preprocessor = ColumnTransformer([('one-hot-encoder', categorical_preprocessor, categorical_columns),
                                  ('standard_scaler', numerical_preprocessor, numerical_columns)])

Now split into testing and training (which randomizes data).

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features_date, target_date, random_state=42)

## Modeling
Use mean squared error, as dates that are much further away are significantly worse.

In [21]:
from sklearn.metrics import mean_squared_error

I'll use normal linear regression first.

In [22]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
pipe_linear = make_pipeline(preprocessor, LinearRegression())
pipe_linear.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(drop='first'),
                                                  ['ranking', 'Type',
                                                   'political_control_state',
                                                   'Region']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['total_population',
                                                   'avg_community_level',
                                                   'county_vote_diff',
                                                   'median_income',
                                                   '2020.student.size',
                                                   'avg_hhsize'])])),
                ('linearregression', Lin

In [23]:
pipe_linear.score(X_test, y_test)

0.4361647632216752

In [24]:
y_pred_linear = pipe_linear.predict(X_test)
mean_squared_error(y_test, y_pred_linear)

1981.728506693466

Compare against dummy estimator for sanity check.

In [197]:
from sklearn.dummy import DummyRegressor
for strategy in ['mean', 'median']:
    dummy_regr = DummyRegressor(strategy=strategy)
    dummy_regr.fit(X_train, y_train)
    y_pred_dummy = dummy_regr.predict(X_test)
    print(f'{strategy}: {dummy_regr.score(X_test, y_test)}, {mean_squared_error(y_test, y_pred_dummy)}')

mean: -0.00040652144083153097, 3516.1586089391376
median: -0.14004089843234668, 4006.9357142857143


My predictions are much better, meaning they have some actual degree of explanatory power.

Though there are no hyperparameters to tune, an example of cross validation is shown as an example.

In [25]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(pipe_linear, X_train, y_train, cv=5)
scores

array([-0.05902675,  0.41527914,  0.24022537,  0.46778222,  0.36731294])

Add regularization--let's try Lasso as some features may note be important, using cross validation to select the hyperparameter.

In [26]:
from sklearn.linear_model import LassoCV
pipe_lasso = make_pipeline(preprocessor, LassoCV())
pipe_lasso.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(drop='first'),
                                                  ['ranking', 'Type',
                                                   'political_control_state',
                                                   'Region']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['total_population',
                                                   'avg_community_level',
                                                   'county_vote_diff',
                                                   'median_income',
                                                   '2020.student.size',
                                                   'avg_hhsize'])])),
                ('lassocv', LassoCV())])

In [27]:
pipe_lasso.score(X_test, y_test)

0.40397416700424804

In [28]:
y_pred_lasso = pipe_lasso.predict(X_test)
mean_squared_error(y_test, y_pred_lasso)

2094.8697543672342

Also not working too well. Try ensemble learning--specifically, a random forest.

In [205]:
from sklearn.ensemble import RandomForestRegressor
pipe_forest = make_pipeline(preprocessor, RandomForestRegressor(random_state=42))
pipe_forest.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('one-hot-encoder',
                                                  OneHotEncoder(drop='first'),
                                                  ['ranking', 'Type',
                                                   'political_control_state',
                                                   'Region']),
                                                 ('standard_scaler',
                                                  StandardScaler(),
                                                  ['total_population',
                                                   'avg_community_level',
                                                   'county_vote_diff',
                                                   'median_income',
                                                   '2020.student.size',
                                                   'avg_hhsize'])])),
                ('randomforestregressor'

In [203]:
def show_metrics(model, X_test, y_test):
    y_pred = model.predict(X_test)
    print(model.score(X_test, y_test))
    print(mean_squared_error(y_test, y_pred))

In [206]:
show_metrics(pipe_forest, X_test, y_test)

0.1839001172674115
2868.3705742857146


Both the coefficient of determination and the mean squared error are relatively high.

So, I'll adjust the hyperparameters by using CV (as my dataset is small so fitting won't be too costly). Note that I could use OOB as well, which I'll do first as the model already calculates it.

In [30]:
pipe_forest.get_params().keys() # possible grid search parameters

dict_keys(['memory', 'steps', 'verbose', 'columntransformer', 'randomforestregressor', 'columntransformer__n_jobs', 'columntransformer__remainder', 'columntransformer__sparse_threshold', 'columntransformer__transformer_weights', 'columntransformer__transformers', 'columntransformer__verbose', 'columntransformer__verbose_feature_names_out', 'columntransformer__one-hot-encoder', 'columntransformer__standard_scaler', 'columntransformer__one-hot-encoder__categories', 'columntransformer__one-hot-encoder__drop', 'columntransformer__one-hot-encoder__dtype', 'columntransformer__one-hot-encoder__handle_unknown', 'columntransformer__one-hot-encoder__sparse', 'columntransformer__standard_scaler__copy', 'columntransformer__standard_scaler__with_mean', 'columntransformer__standard_scaler__with_std', 'randomforestregressor__bootstrap', 'randomforestregressor__ccp_alpha', 'randomforestregressor__criterion', 'randomforestregressor__max_depth', 'randomforestregressor__max_features', 'randomforestregres

In [207]:
from sklearn.model_selection import GridSearchCV
param_grid = [
    {'randomforestregressor__n_estimators': [25, 75, 100, 150, 200],
     'randomforestregressor__max_depth': [None, 5, 10, 15],
     'randomforestregressor__max_features': [None, 0.25, 0.5, 0.75], # Note I have 10 features (when they're not encoded)
     'randomforestregressor__max_leaf_nodes': [None, 2, 5, 10], 
     'randomforestregressor__max_samples': [None, 25, 50, 75] # 104 samples in training data -- 84 in validation
    }
]

I'm copying code from [Ben Reiniger's answer from StackExchange](https://datascience.stackexchange.com/questions/66216/gridsearch-without-cv/66238#66238) below. Need to create a metric that measures OOB then pass it to ```GridSearchCV```.

In [208]:
from sklearn.model_selection import PredefinedSplit
cv = PredefinedSplit([-1]*(X_train.shape[0]-1) + [0])
for (train, test) in cv.split(X_train, y_train):
    print(len(train), len(test))

103 1


In [210]:
def oob_scorer(estimator, X, y):
    """
    Get oob score where RandomForest is 1st element in Pipeline
    """
    return estimator[1].oob_score_
pipe_forest_oob = GridSearchCV(estimator=make_pipeline(preprocessor, RandomForestRegressor(random_state=42, oob_score=True)), 
                               param_grid=param_grid, scoring=oob_scorer, cv=cv)
pipe_forest_oob.fit(X_train, y_train)

GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ..., -1,  0])),
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('one-hot-encoder',
                                                                         OneHotEncoder(drop='first'),
                                                                         ['ranking',
                                                                          'Type',
                                                                          'political_control_state',
                                                                          'Region']),
                                                                        ('standard_scaler',
                                                                         StandardScaler(),
                                                                         ['total_population',
                                         

In [211]:
pipe_forest_oob.best_params_

{'randomforestregressor__max_depth': 5,
 'randomforestregressor__max_features': 0.75,
 'randomforestregressor__max_leaf_nodes': 10,
 'randomforestregressor__max_samples': None,
 'randomforestregressor__n_estimators': 150}

In [212]:
show_metrics(pipe_forest_oob, X_test, y_test)

0.2651624407049967
2866.8761660137266


So, there's a small improvement using grid search and oob error. Note that takes a short amount of time as no validation set is needed--the parts of the training data that are not used in creating each weak learner are used to get a fairly unbiased estimate.

Now do CV, which should be similar but take longer, as training and evaluation is needed for each fold.

In [214]:
pipe_forest_grid = GridSearchCV(make_pipeline(preprocessor, RandomForestRegressor(random_state=42)), param_grid)
pipe_forest_grid.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('one-hot-encoder',
                                                                         OneHotEncoder(drop='first'),
                                                                         ['ranking',
                                                                          'Type',
                                                                          'political_control_state',
                                                                          'Region']),
                                                                        ('standard_scaler',
                                                                         StandardScaler(),
                                                                         ['total_population',
                                                                          'avg_community_level',
                 

In [215]:
pipe_forest_grid.best_params_

{'randomforestregressor__max_depth': None,
 'randomforestregressor__max_features': 0.75,
 'randomforestregressor__max_leaf_nodes': 10,
 'randomforestregressor__max_samples': 75,
 'randomforestregressor__n_estimators': 75}

In [216]:
pipe_forest_grid.score(X_test, y_test)

0.1699791929868104

In [217]:
y_pred_forest_grid = pipe_forest_grid.predict(X_test)
mean_squared_error(y_test, y_pred_forest_grid)

2917.298861641467

Regular grid search takes a long time to run. My dataset is fairly small, so I'm ok with this. However, if it was larger, I would use either ```RandomizedSearchCV```,  ```HalvingGridSearchCV``` or ```HalvingRandomSearchCV```. 

Here I'll use ```HalvingGridSearchCV```. This method starts by evaluating each model with a small amount of resources (e.g., a small number of ```n_estimators``` or samples) and chooses the best models from there. It increases the resources by multiplying it by ```factor``` and decreases the number of candidate models by dividing it by ```factor```. We will set ```min_resources='exhaust'``` so that the last iteration uses all resources possible, set by ```max_resources```.

In [123]:
pipe_forest_no_oob = make_pipeline(preprocessor, RandomForestRegressor())

In [124]:
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingGridSearchCV
param_grid_halving = [
    {'randomforestregressor__max_depth': [None, 5, 10, 15],
     'randomforestregressor__max_features': [None, 0.25, 0.5, 0.75], # Note I have 10 features (when they're not encoded)
     'randomforestregressor__max_leaf_nodes': [None, 2, 5, 10], 
     'randomforestregressor__max_samples': [None, 25, 50, 75] # 104 samples in training data -- 84 in validation
    }
]
pipe_forest_halving_grid = HalvingGridSearchCV(pipe_forest_no_oob, param_grid_halving, cv=5, factor=2,
                                               resource='randomforestregressor__n_estimators', max_resources=200)
pipe_forest_halving_grid.fit(X_train, y_train)

HalvingGridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                               ColumnTransformer(transformers=[('one-hot-encoder',
                                                                                OneHotEncoder(drop='first'),
                                                                                ['ranking',
                                                                                 'Type',
                                                                                 'political_control_state',
                                                                                 'Region']),
                                                                               ('standard_scaler',
                                                                                StandardScaler(),
                                                                                ['total_population',
                                            

In [125]:
pipe_forest_halving_grid.best_params_

{'randomforestregressor__max_depth': 5,
 'randomforestregressor__max_features': 0.75,
 'randomforestregressor__max_leaf_nodes': 10,
 'randomforestregressor__max_samples': None,
 'randomforestregressor__n_estimators': 128}

In [126]:
pipe_forest_halving_grid.score(X_test, y_test)

0.1757530795905723

In [127]:
y_pred_forest_halving_grid = pipe_forest_halving_grid.predict(X_test)
mean_squared_error(y_test, y_pred_forest_halving_grid)

2897.0052103569706

Using halving grid search with my specified parameters produces a model that's about equivalent to my original random forest. Change the parameters a little.

In [133]:
param_grid_halving_new = [
    {'randomforestregressor__max_depth': [8, 9, 10, 11, 12],
     'randomforestregressor__max_features': [0.5, 0.6, 0.75, 0.8, 0.9, 1.0], # Note I have 10 features (when they're not encoded)
     'randomforestregressor__max_leaf_nodes': [4, 5, 7, 8], 
     'randomforestregressor__max_samples': [None, 70, 75, 80] # 104 samples in training data -- 84 in validation
    }
]
pipe_forest_halving_grid_new = HalvingGridSearchCV(pipe_forest_no_oob, param_grid_halving_new, cv=5, factor=2,
                                               resource='randomforestregressor__n_estimators', max_resources=256)
pipe_forest_halving_grid_new.fit(X_train, y_train)

HalvingGridSearchCV(estimator=Pipeline(steps=[('columntransformer',
                                               ColumnTransformer(transformers=[('one-hot-encoder',
                                                                                OneHotEncoder(drop='first'),
                                                                                ['ranking',
                                                                                 'Type',
                                                                                 'political_control_state',
                                                                                 'Region']),
                                                                               ('standard_scaler',
                                                                                StandardScaler(),
                                                                                ['total_population',
                                            

In [134]:
pipe_forest_halving_grid.best_params_

{'randomforestregressor__max_depth': 5,
 'randomforestregressor__max_features': 0.75,
 'randomforestregressor__max_leaf_nodes': 10,
 'randomforestregressor__max_samples': None,
 'randomforestregressor__n_estimators': 128}

In [135]:
pipe_forest_halving_grid.score(X_test, y_test)

0.1757530795905723

In [136]:
y_pred_forest_halving_grid = pipe_forest_halving_grid.predict(X_test)
mean_squared_error(y_test, y_pred_forest_halving_grid)

2897.0052103569706

Using halving grid actually turned out worse on my test set than my base random forest model, but running my original grid search turned out best and ended up not taking too long (not over 15min).

Based on these results, I don't think using a random forest is best. Maybe I could improve it more, but based on my metrics, I am far off from the results with linear regression. So, original linear regression seems the best, likely due to our small sample and its good generalization. 

I'll try one more regression model before moving on -- SVM regression

*Note*: My test set is not a true test set if we define it to be an unbiased estimate of the model's performance on new data. This is because I've used it to choose the best model--each model can be considered a hyperparameter that I'm tuning, so the test set is no longer a truly unbiased estimate.

See these answers for more:
- https://stats.stackexchange.com/questions/540454/is-it-wrong-to-compare-multiple-models-on-the-same-test-set-and-choose-the-best
- https://datascience.stackexchange.com/questions/43210/can-i-use-the-test-dataset-to-select-a-model 

**Maybe should output a rank (i.e., which universities acted first)?**

## Sources
- https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html
- https://towardsdatascience.com/7-of-the-most-commonly-used-regression-algorithms-and-how-to-choose-the-right-one-fc3c8890f9e3
- https://scikit-learn.org/stable/model_selection.html
- https://towardsdatascience.com/the-5-classification-evaluation-metrics-you-must-know-aa97784ff226